In [1]:
import ee
from utils import *

ee.Authenticate()
ee.Initialize(project='dse-staff')

geo_ops = GeometryOperations()
img_ops = ImageOperations()
stats_ops = StatsOperations()
feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6
/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:209: DeprecationWarning: 

Attention required for JRC/GSW1_0/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_0_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:209: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)


In [ ]:
water_mask = ee.Image("JRC/GSW1_4/GlobalSurfaceWater") 
max_error = 30

def load_protected_areas_by_ids(wdpa_ids_list):
    protected_areas = ee.FeatureCollection('WCMC/WDPA/current/polygons')
    pa_collection = protected_areas.filter(ee.Filter.inList('WDPA_PID', wdpa_ids_list))
    return pa_collection

def mask_water(feat):
    water_no_holes = water_mask.select('max_extent') \
        .focalMax(radius=30, units='meters', kernelType='square') \
        .focalMin(radius=30, units='meters', kernelType='square')
    water_vect = water_no_holes.reduceToVectors(
        reducer=ee.Reducer.countEvery(),
        geometry=feat.buffer(1000),
        scale=30,
        maxPixels=1e10,
        geometryType='polygon',
        eightConnected=False
    )
    return feat.difference(water_vect.geometry(), maxError=max_error)

def buffer_polygon(feat, buffer_distance):
    """Create buffer around polygon"""
    feat = ee.Feature(feat)
    out = feat.buffer(buffer_distance).geometry()
    inn = feat.buffer(-buffer_distance).geometry()
    geom = out.difference(inn, max_error)
    return geom

def make_ribbons(feat):
    km = 1000
    aoi = mask_water(buffer_polygon(feat, 5 * km))  # outer buffer (5 km)
    inner = mask_water(buffer_polygon(feat, -1 * km)) 
    outer = aoi.difference(inner, maxError=max_error)
    fid = feat.get("WDPA_PID")
    inner_f = ee.Feature(inner, {"WDPA_PID": fid, "zone": "inner"})
    outer_f = ee.Feature(outer, {"WDPA_PID": fid, "zone": "outer"})
    return ee.FeatureCollection([inner_f, outer_f])

In [6]:
serengeti = load_protected_areas_by_ids(['916']).first()
ribbons = make_ribbons(serengeti)
inner = ribbons.filter(ee.Filter.eq('zone', 'inner'))
outer = ribbons.filter(ee.Filter.eq('zone', 'outer'))

Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(serengeti, 9)
Map.addLayer(serengeti, {'color': 'red'}, 'serengeti National Park')
Map.addLayer(inner, {'color': 'green'}, 'Inner Ribbon (1 km)')
Map.addLayer(outer, {'color': 'blue'}, 'Outer Ribbon (1–5 km)')
Map

Map(center=[-2.3382991604579066, 34.78479649969417], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
print(ribbons.getInfo())

{'type': 'FeatureCollection', 'columns': {'WDPA_PID': 'String', 'system:index': 'String', 'zone': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '0', 'properties': {'WDPA_PID': '916', 'zone': 'inner'}}, {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '1', 'properties': {'WDPA_PID': '916', 'zone': 'outer'}}]}


In [ ]:
# Expand each park into 3 features
expanded = parks.map(make_ribbons).flatten()

modis = ee.ImageCollection("MODIS/006/MOD13Q1").select("NDVI")
composite = modis.filterDate('2019-01-01', '2019-12-31').median()
gradient = img_ops.get_gradient_magnitude(composite)

# Zonal stats per park-zone
stats = gradient.reduceRegions(
    collection=expanded,
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), "", True),
    scale=250,
    tileScale=2
)

# Export to Cloud Storage
task = ee.batch.Export.table.toCloudStorage(
    collection=stats,
    description="ndvi_zonal_ribbons_2019",
    bucket="dse-staff/protected_areas/",
    fileNamePrefix="ndvi_zonal_ribbons_2019",
    fileFormat="CSV"
)
task.start()
